In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [1]:
import sys
from google.colab import drive

drive.mount('/content/mydrive', force_remount=True)
sys.path.insert(0,"/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/")

Mounted at /content/mydrive


In [4]:
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

0

In [5]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import xml.etree.ElementTree as ET
from google.colab import drive
import os
import pandas as pd
from torchvision import transforms
import os
import pandas as pd
from torchvision import transforms
from PIL import Image




In [8]:
import os
data_dir = '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/'
os.listdir(data_dir)

['f6230df97600c1a1e3c63d72900af19f.csv',
 'f66aa5f00ba96133629d46a6a6db09e8.csv',
 'f5d35995b3a01a2178ee6e6541f1d5c8.csv',
 'f66feb45321aa64c4545cab4e79f8839.jpg',
 'f5e6d361dc9398399eff41ef397f2425.csv',
 'f5f01d44ecd56b5b3253eb35c266870e.csv',
 'f64dc4d86868c6440e5f2b44cc4aea6c.csv',
 'f632da330108c076e77dfbab1793dbf0.csv',
 'f65c1cdc5597c27fb9158d8254863000.jpg',
 'f635b3292328c0f8ca0eb3c3d160a376.jpg',
 'f5e6007d2207a0f932ae51bb7a9e3e94.jpg',
 'f5d25f33689e16803b1d9975f4387873.csv',
 'f5c9daf55818c17de2e8999682108eab.csv',
 'f66aa5f00ba96133629d46a6a6db09e8.jpg',
 'f69d995e4719eefe7b778ffcf4525c23.jpg',
 'f69cf468fc7e3ad9a6c357398e25acba.jpg',
 'f6947b372d9d3b35f151811bd865a25a.jpg',
 'f6a58733f3ba1b512333dfaa65613807.jpg',
 'f6aa6665b28f0cdae59835ac9b8a48b0.csv',
 'f6aa6665b28f0cdae59835ac9b8a48b0.jpg',
 'f6940e48271152eee845e29793822aab.csv',
 'f6b6a569b2548f7c0e25e1046eeb83fd.csv',
 'f6c64ecf17f9f2386989107acd448679.csv',
 'f6ae241c65a79a127f76a7b2da14d6ab.jpg',
 'f6b4fb91b47c29

In [9]:

count = 1
annotation_paths  = []
for f in os.listdir(data_dir):
    if f.endswith('.csv'):
        count += 1
        if(count%1000 == 0):
          print(count)
        annotation_paths.append(os.path.join(data_dir, f))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [14]:
import dask.dataframe
print(len(annotation_paths))
imgFiles = []
boxes = []
labels = []
count = 0
for f in annotation_paths:
    count += 1
    if(count%100 == 0):
      print(count)
    #data = dask.dataframe.read_csv()
    df = dask.dataframe.read_csv(f)
    for index, row in df.iterrows():
      class_label = row['class']
      xmin = row['xmin']
      ymin = row['ymin']
      xmax = row['xmax']
      ymax = row['ymax']
      boxes.append([xmin, ymin, xmax, ymax])
      labels.append(class_label)
      imgFiles.append(row['filename'] + ".jpg")

print(len(boxes))
print(len(labels))
print(len(imgFiles))

12291
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
19746
19746
19746


In [26]:
from sklearn.preprocessing import LabelEncoder
from torchvision import tv_tensors
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
class CustomDataset(Dataset):
    def __init__(self, img_files, boxes, labels, transforms):
        self.img_files = img_files
        self.boxes = boxes
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join('/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset', self.img_files[idx])
        box = self.boxes[idx]
        label = self.labels[idx]

        # Load image
        image = Image.open(img_path).convert("RGB")
        #image = image.resize(self.target_size, Image.BILINEAR)

        # Convert box to tensor
        box = torch.tensor([box], dtype=torch.float32)

        # Convert label to tensor
        label = torch.tensor([label], dtype=torch.int64)
        image = tv_tensors.Image(image)
        # Create target dictionary
        target = {
            'boxes': box,
            'labels': label,
            'image_id': torch.tensor([idx]),  # Add image ID for multiclass detection
            'area': torch.tensor([(box[0][2] - box[0][0]) * (box[0][3] - box[0][1])], dtype=torch.float32),
            'iscrowd': torch.tensor([0], dtype=torch.int64)
        }

        if self.transforms is not None:
            image, target = self.transforms(image, target)


        return image, target

In [33]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    num_classes = 47  # 1 class (person) + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model


from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    # if train:
    #     transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())

    return T.Compose(transforms)

# get the model using our helper function
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model_instance_segmentation(43)
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [34]:
dataset = CustomDataset(imgFiles, boxes, labels, get_transform(train=True))
dataset_test = CustomDataset(imgFiles, boxes, labels, get_transform(train=True))

In [35]:
def collate_fn(batch):
    return tuple(zip(*batch))
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)


In [36]:

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    device_count = torch.cuda.device_count()
    print(f"Number of available CUDA devices: {device_count}")

    # Print properties of each CUDA device
    for i in range(device_count):
        device = torch.cuda.get_device_properties(i)
        print(f"Device {i}: {device.name}, CUDA Capability {device.major}.{device.minor}")
else:
    print("CUDA is not available.")


CUDA is available!
Number of available CUDA devices: 1
Device 0: Tesla T4, CUDA Capability 7.5


In [37]:
# move model to the right device
device = "cuda" if torch.cuda.is_available() else "cpu"
for p in model.backbone.parameters():
  p.requires_grad = False

# freeze the fc6 layer in roi_heads
for p in model.roi_heads.box_head.fc6.parameters():
  p.requires_grad = False
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)


In [38]:
from utils import MetricLogger
from engine import train_one_epoch, evaluate

# let's train it for 1 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

print("That's it!")

Epoch: [0]  [   0/4937]  eta: 3:17:12  lr: 0.000010  loss: 3.8204 (3.8204)  loss_classifier: 3.6905 (3.6905)  loss_box_reg: 0.1098 (0.1098)  loss_objectness: 0.0123 (0.0123)  loss_rpn_box_reg: 0.0077 (0.0077)  time: 2.3967  data: 1.6712  max mem: 8613
Epoch: [0]  [  10/4937]  eta: 1:22:15  lr: 0.000060  loss: 3.7032 (3.6569)  loss_classifier: 3.5811 (3.5098)  loss_box_reg: 0.1098 (0.1143)  loss_objectness: 0.0187 (0.0256)  loss_rpn_box_reg: 0.0037 (0.0071)  time: 1.0016  data: 0.3390  max mem: 8613
Epoch: [0]  [  20/4937]  eta: 1:11:28  lr: 0.000110  loss: 3.1837 (3.0628)  loss_classifier: 3.0401 (2.9159)  loss_box_reg: 0.1055 (0.1139)  loss_objectness: 0.0185 (0.0240)  loss_rpn_box_reg: 0.0064 (0.0090)  time: 0.7960  data: 0.1181  max mem: 8613
Epoch: [0]  [  30/4937]  eta: 1:09:53  lr: 0.000160  loss: 1.4725 (2.3681)  loss_classifier: 1.3548 (2.2180)  loss_box_reg: 0.1093 (0.1196)  loss_objectness: 0.0162 (0.0227)  loss_rpn_box_reg: 0.0058 (0.0078)  time: 0.7736  data: 0.0377  max me

/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:272: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  ar = np.asanyarray(ar)


AssertionError: ignored